In [1]:
#imports the CSV as well as the dependencies and sets the perameter for Null fields 
#for filling later and shows the size of the array
import pandas as pd
import numpy as np
buenos_aires_df = pd.read_csv('buenos_aires_dataframe.csv', sep=";")
madrid_df = pd.read_csv('madrid_dataframe.csv', sep=";")
montevideo_df = pd.read_csv('montevideo_dataframe.csv', sep=";")

pd.options.display.max_rows = 20

In [6]:
from pysentimiento import create_analyzer
import spacy
from spacy import displacy
import pandas as pd
import numpy as np
import nltk
import pickle
from collections import OrderedDict, Counter
# Stanza for sentiment analysis

stopwords = nltk.corpus.stopwords.words('spanish')
stopwords.append(" ")
stopwords.append(",")
stopwords.append(":")
stopwords.append(";")
stopwords.append(".")
stopwords.append("!")
stopwords.append("?")
stopwords.append("'")
stopwords.append("¿")
stopwords.append("\"")
stopwords.append("si")

analyzer = create_analyzer(task="sentiment", lang="es")
analyzer = pickle.load( open( "sent_analyzer.p", "rb" ) )
nlp = pickle.load( open( "pos_tagger.p", "rb" ) )

def parseCSV(filePath):
    # CVS Column Names
    col_names = ['Index','File', 'Name', 'Singer', 'Place', 'Year', 'Genre', 'LyricNumber', 'Composer', 'Sex', 'Lyrics', 'Text', 'Words']
    # Use Pandas to parse the CSV file
    csvData = pd.read_csv(filePath,sep=";",names=col_names, header=0)
    # Get content_words and get a list of each unique word in a song
    csvData["content_words"] = csvData["Words"].apply(lambda x: [item for item in str(x).replace("\"","").replace("'","").replace("[","").replace(" ", "").replace("]","").split(",") if item not in stopwords])
    csvData["unique_words"] = csvData["content_words"].apply(lambda x: np.delete(np.unique(x), np.where(np.unique(x) == '')))
    # Uses spacy for labeling pos for content words
    pos = []
    for text in csvData["Text"]:
        doc = nlp(text)
        pos.append(doc)
    csvData["pos"] = pos 
    csvData["content_pos"] = csvData["pos"].apply(lambda x: [item for item in x if str(item) not in stopwords])
    csvData["content_pos_count"] = csvData["content_pos"].apply(lambda x: OrderedDict(Counter([item.pos_  for item in x]).most_common()))
    csvData["POS"] = csvData["content_pos"].apply(lambda x: [str(item.lemma_ + ": " + item.pos_ + " " + item.dep_)  for item in x])
    
    # Uses pysentimentio for labeling the song as pos, neg, neu and give percentages of those
    sentiment_out = []
    sentiment_neg = []
    sentiment_neu = []
    sentiment_pos = []
    for text in csvData["Text"]:
        res = analyzer.predict(text)
        sentiment_out.append(res.output)
        sentiment_neg.append(res.probas["NEG"])
        sentiment_neu.append(res.probas["NEU"])
        sentiment_pos.append(res.probas["POS"])
    csvData['Sentiment_Output'] = sentiment_out
    csvData['Sentiment_Neg_Probability'] = sentiment_neg
    csvData['Sentiment_Neu_Probability'] = sentiment_neu
    csvData['Sentiment_Pos_Probability'] = sentiment_pos
    return csvData
csv1 = parseCSV("buenos_aires_dataframe.csv")
csv2 = parseCSV("madrid_dataframe.csv")
csv3 = parseCSV("montevideo_dataframe.csv")
csvs = [csv1,csv2,csv3]
all_df = pd.concat(csvs)
trimmed = all_df.drop(['content_pos'], axis=1)
# all_df = all_df.rename({'Unnamed: 0': 'Index'}, axis=1)
pickle.dump( trimmed, open( "lyrics_corpus.p", "wb" ) )

loading configuration file config.json from cache at C:\Users\joana/.cache\huggingface\hub\models--pysentimiento--robertuito-sentiment-analysis\snapshots\12e030859ce19539e24b486ac84ffebb9b68ecf1\config.json
Model config RobertaConfig {
  "_name_or_path": "pysentimiento/robertuito-sentiment-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NEG",
    "1": "NEU",
    "2": "POS"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NEG": 0,
    "NEU": 1,
    "POS": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type":

In [37]:
# trimmed = all_df.drop(['Lyrics','content_pos','Words'], axis=1)
# # all_df = all_df.rename({'Unnamed: 0': 'Index'}, axis=1)
# pickle.dump( trimmed, open( "lyrics_corpus.p", "wb" ) )
all_df = pickle.load( open( "lyrics_corpus.p", "rb" ) )
all_df

,Index,File,Name,Singer,Place,Year,Genre,LyricNumber,Composer,Sex,...,Words,content_words,unique_words,pos,content_pos_count,POS,Sentiment_Output,Sentiment_Neg_Probability,Sentiment_Neu_Probability,Sentiment_Pos_Probability
0,0,17. Rambla Sur.docx,RAMBLA SUR,LARBANOIS & CARRERO,MONTEVIDEO,1988,POPULAR,17,LARBANOIS & CARRERO,MALE,...,"['a', 'dónde', 'se', 'han', 'ido', 'mis', 'die...","[dónde, ido, dieciocho, años, primera, quincen...","[31, adoquín, ahí, aire, alma, anda, apagó, aq...","(A, dónde, se, han, ido, mis, dieciocho, años,...","{'NOUN': 63, 'VERB': 32, 'PRON': 12, 'PROPN': ...","[a: ADP case, dónde: PRON obl, ir: VERB ROOT, ...",NEU,0.327286,0.644070,0.028643
1,1,34. Montevideo.docx,MONTEVIDEO,ANTONIO BIRABENT,MONTEVIDEO,2003,ROCK,34,ANTONIO BIRABENT,MALE,...,"['crucé', 'el', 'río', 'tenía', 'tanto', 'frío...","[crucé, río, frío, busqué, abrigo, gente, enfr...","[abrigo, acusa, agua, ahora, ahí, aires, allí,...","(Crucé, el, río, tenía, tanto, frío, busqué, a...","{'VERB': 65, 'NOUN': 49, 'PROPN': 35, 'ADJ': 3...","[crucé: VERB ccomp, río: NOUN obj, frío: NOUN ...",NEU,0.028611,0.597153,0.374235
2,2,70. Yo nací en Jacinto Vera.docx,YO NACI EN JACINTO VERA,DANIEL VIGLIETTI,MONTEVIDEO,1968,POPULAR,70,LIBER FALCO-DANIEL VIGLIETTI,MALE,...,"['yo', 'nací', 'en', 'jacinto', 'vera', 'qué',...","[nací, jacinto, vera, barrio, jacinto, vera, r...","[ah, aparecía, barrio, blanca, corría, dentro,...","(Yo, nací, en, Jacinto, Vera, qué, barrio, Jac...","{'NOUN': 15, 'PROPN': 8, 'VERB': 6, 'ADP': 4, ...","[yo: PRON nsubj, nací: PROPN flat, Jacinto: PR...",NEU,0.087198,0.757031,0.155772
3,3,62. Retirada 1991.docx,RETIRADA 1991,CONTRAFARSA,MONTEVIDEO,1991,MURGA,62,PITUFO LOMBARDO,MALE,...,"['contrafarsa', 'es', 'voz', 'de', 'esquina', ...","[contrafarsa, voz, esquina, aire, febrero, sue...","[abrace, abre, abuelas, abuelos, acaricia, aco...","(Contrafarsa, es, voz, de, esquina, que, en, e...","{'NOUN': 176, 'VERB': 109, 'PROPN': 45, 'ADJ':...","[Contrafarsa: PROPN nsubj, voz: NOUN ROOT, esq...",POS,0.034931,0.462520,0.502550
4,4,10. Alegris.docx,ALEGRIS,LA TABARE,MONTEVIDEO,1992,ROCK,10,TABARE RIVERO,MALE,...,"['voy', 'por', 'ahí', 'somos', 'tantos', 'dese...","[voy, ahí, tantos, desencuentros, callejeando,...","[18, agarro, agenda, ahi, ahí, alegria, alegri...","(Voy, por, ahí, somos, tantos, desencuentros, ...","{'NOUN': 55, 'VERB': 48, 'ADJ': 16, 'PROPN': 1...","[ir: AUX aux, ahí: ADV advmod, tanto: DET det,...",NEG,0.779223,0.205093,0.015683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,89,B.A. 38 No tan Buenos Aires.docx,NO TAN BUENOS AIRES,ANDRES CALAMARO,BUENOS AIRES,1996,ROCK,38,ANDRES CALAMARO,MALE,...,"['ya', 'siento', 'que', 'estoy', 'radiante', '...","[siento, radiante, volver, cuenta, diamante, c...","[acá, además, afición, agitar, aire, aires, al...","(Ya, siento, que, estoy, radiante, por, volver...","{'NOUN': 84, 'VERB': 76, 'ADJ': 39, 'PROPN': 2...","[ya: ADV advmod, sentir: VERB ROOT, radiante: ...",POS,0.005585,0.173783,0.820632
90,90,B.A. 94 CABA.docx,CABA,PACHA GONZALEZ,BUENOS AIRES,2012,???,94,PACHA GONZALEZ,MALE,...,"['son', 'tan', 'suaves', 'estas', 'avenidas', ...","[tan, suaves, avenidas, chicas, atrevidas, , s...","[abombada, abrigo, aires, alma, alumbrando, am...","( , Son, tan, suaves, estas, avenidas, y, las,...","{'NOUN': 78, 'VERB': 43, 'ADV': 15, 'ADJ': 15,...","[ser: AUX cop, tanto: ADV advmod, suave: ADJ a...",NEG,0.817872,0.160108,0.022020
91,91,B.A. 47 Buenos Aires.docx,BUENOS AIRES,FITO PAEZ,BUENOS AIRES,1998,ROCK,47,FITO PAEZ,MALE,...,"['en', 'buenos', 'aires', 'brilla', 'el', 'sol...","[buenos, aires, brilla, sol, par, pibes, , esq...","[(cuando, absurda, acarician, acechan, agencia...","(En, Buenos, Aires, brilla, el, sol, Y, un, pa...","{'NOUN': 89, 'PROPN': 85, 'VERB': 60, 'ADP': 2...","[en: ADP case, Buenos: PROPN obl, Aires: PROPN...",NEU,0.034670,0.625425,0.339905
92,92,B.A. 72 Cafetin de Buenos Aires.docx,CAFETIN DE BUENOS AIRES,ENRIQUE SANTE

In [40]:
word_counts = []
sent_counts = []
places = all_df['Place'].unique()
sent_counts.append(sent_song_count( 'all'))
word_counts.append(word_across_song_count( 10, 'all'))
for city in places:
    word_counts.append(word_across_song_count( 10, city))
    sent_counts.append(sent_song_count( city))
len(sent_counts)

3

In [55]:
csvs = [csv1]
all_df = pd.concat(csvs)
trimmed = all_df.drop(['content_pos'], axis=1)
pickle.dump( trimmed, open( "lyrics_corpus.p", "wb" ) )

In [20]:
from collections import Counter
from collections import OrderedDict
wc_threshold = 10


cities = all_df['Place'].unique()
city_words = []
wc = Counter()
for city in cities:
    print(city)
    city_df = all_df.loc[all_df['Place'] == city]
    uniq_words = city_df["unique_words"].to_numpy()
    uniq_words = np.hstack(uniq_words)
    wc.update(Counter(uniq_words))
wc = Counter(el for el in wc.elements() if wc[el] >= wc_threshold)
word_count = sorted(wc.items(), key=lambda item: (-item[1], item[0]))
all_words = OrderedDict(word_count)
all_words

MONTEVIDEO


OrderedDict([('siempre', 25),
             ('mar', 21),
             ('montevideo', 20),
             ('ciudad', 19),
             ('gente', 19),
             ('noche', 19),
             ('va', 17),
             ('barrio', 16),
             ('vez', 16),
             ('calle', 15),
             ('corazón', 15),
             ('hoy', 15),
             ('tiempo', 15),
             ('vida', 15),
             ('van', 14),
             ('voy', 14),
             ('día', 13),
             ('igual', 13),
             ('vieja', 13),
             ('allá', 12),
             ('viejo', 12),
             ('viento', 12),
             ('alma', 11),
             ('bien', 11),
             ('hace', 11),
             ('luna', 11),
             ('nunca', 11),
             ('sol', 11),
             ('tan', 11),
             ('vivir', 11),
             ('amor', 10),
             ('aquí', 10),
             ('cada', 10),
             ('da', 10),
             ('lugar', 10),
             ('playa', 10),
          

In [23]:
all_df['Place'].unique()

array(['MONTEVIDEO'], dtype=object)

In [56]:
from collections import Counter
from collections import OrderedDict

def word_across_song_count(threshold, city):
    wc = Counter()
    if city == 'all':
        cities = all_df['Place'].unique()
        for city in cities:
            city_df = all_df.loc[all_df['Place'] == city]
            uniq_words = city_df["unique_words"].to_numpy()
            uniq_words = np.hstack(uniq_words)
            wc.update(Counter(uniq_words))
        wc = Counter(el for el in wc.elements() if wc[el] >= threshold)
        word_count = sorted(wc.items(), key=lambda item: (-item[1], item[0]))
        all_words = OrderedDict(word_count)
        return all_words
    else:
        city_df = all_df.loc[all_df['Place'] == city]
        uniq_words = city_df["unique_words"].to_numpy()
        uniq_words = np.hstack(uniq_words)
        wc.update(Counter(uniq_words))
        wc = Counter(el for el in wc.elements() if wc[el] >= threshold)
        word_count = sorted(wc.items(), key=lambda item: (-item[1], item[0]))
        all_words = OrderedDict(word_count)
        return all_words

wc_all = word_across_song_count(10, 'all')
wc_ba = word_across_song_count(10, 'BUENOS AIRES')
# wc_mad = word_across_song_count(10, 'MADRID')
# wc_mv = word_across_song_count(10, 'MONTEVIDEO')
# word_counts = [wc_ba, wc_mad, wc_mv, wc_all]
word_counts = [ wc_ba, wc_all]
pickle.dump( word_counts, open( "word_counts.p", "wb" ) )

In [57]:
def sent_song_count(city):
    if city == 'all':
        songs = all_df['Sentiment_Output'].values.tolist()
        sentcount = Counter(songs)
        sent_count = sorted(sentcount.items(), key=lambda item: (-item[1], item[0]))
        sentiment_count = OrderedDict(sent_count)
        return sentiment_count
    else:
        city_df = all_df.loc[all_df['Place'] == city]
        songs = city_df["Sentiment_Output"].values.tolist()
        sentcount = Counter(songs)
        sent_count = sorted(sentcount.items(), key=lambda item: (-item[1], item[0]))
        sentiment_count = OrderedDict(sent_count)
        return sentiment_count
sc_all = sent_song_count("all")
sc_ba = sent_song_count("BUENOS AIRES")
# sc_mad = sent_song_count("MADRID")
# sc_mv = sent_song_count("MONTEVIDEO")
# sent_counts = [sc_ba, sc_mad, sc_mv, sc_all]
sent_counts = [sc_ba, sc_all]
pickle.dump( sent_counts, open( "sent_counts.p", "wb" ) )

In [118]:
all_df

,Index,File,Name,Singer,Place,Year,Genre,LyricNumber,Composer,Sex,...,Words,content_words,unique_words,pos,content_pos,POS,Sentiment_Output,Sentiment_Neg_Probability,Sentiment_Neu_Probability,Sentiment_Pos_Probability
0,0,B.A. 51 Agua en Buenos Aires.docx,AGUA EN BUENOS AIRES,DIVIDIDOS,BUENOS AIRES,1995,ROCK-POP?,51,DIEGO ARNERO-RICARDO MOLLO,MALE,...,"['sale', 'un', 'bote', 'a', 'sufrir', 'no', 'e...","[sale, bote, sufrir, venecia, camalote, va, fl...","[agua, aires, anfibio, arrastrado, atucha, bot...","(Sale, un, bote, a, sufrir, No, es, como, Vene...","[Sale, bote, sufrir, No, Venecia, Camalote, va...","[salir: VERB advcl, bote: NOUN obj, sufrir: VE...",NEG,0.732685,0.250028,0.017287
1,1,B.A. 58 Idolo de los quemados.docx,IDOLO DE LOS QUEMADOS,LEON GIECO,BUENOS AIRES,2001,ROCK,58,LEON GIECO,MALE,...,"['soy', 'mezcla', 'de', 'baguala,', 'internet'...","[mezcla, baguala, , internet, tango, viejo, he...","[2001, 3000, 411, aconsejó, aires, alas, alcan...","(Soy, mezcla, de, baguala, ,, internet, y, tan...","[Soy, mezcla, baguala, internet, tango, viejo,...","[ser: AUX cop, mezcla: NOUN ROOT, baguala: NOU...",NEU,0.416313,0.519488,0.064199
2,2,B.A. 25 Con la frente marchita.docx,CON LA FRENTE MARCHITA,JOAQUIN SABINA,BUENOS AIRES,1990,POP-BANDONEON?,25,JOAQUIN SABINA,MALE,...,"['sentados', 'en', 'corro', 'merendábamos', 'b...","[sentados, corro, merendábamos, besos, porros,...","[80, adiós, agüita, aires, amor, andaluz, aque...","(Sentados, en, corro, Merendábamos, besos, y, ...","[Sentados, corro, Merendábamos, besos, porros,...","[sentado: ADJ nsubj, corro: NOUN advcl, Merend...",NEU,0.200022,0.635468,0.164511
3,3,B.A. 15 Los locos de Buenos Aires.docx,LOS LOCOS DE BUENOS AIRES,ALEJANDRO DEL PRADO,BUENOS AIRES,1984,POP,15,ALEJANDRO DEL PRADO,MALE,...,"['uno', 'está', 'solo', 'y', 'espera,', 'otro'...","[solo, espera, , cree, manera, , ciego, locura...","[40, actor, aires, ama, amenazante, andan, art...","(Uno, está, solo, y, espera, ,, Otro, cree, a,...","[Uno, solo, espera, Otro, cree, manera, Otro, ...","[Uno: PROPN nsubj, solo: ADJ advcl, esperar: V...",NEU,0.475670,0.490647,0.033683
4,4,B.A. 65 Oeste Cercano.docx,OESTE CERCANO,ANTONIO BIRABENT,BUENOS AIRES,2005,POP,65,ANTONIO BIRABENT,MALE,...,"['barrios', 'interminables,', 'la', 'ciudad', ...","[barrios, interminables, , ciudad, fin, hace, ...","[adivino., aldea, almohada, arteria, asfalto.,...","( , Barrios, interminables, ,, la, ciudad, no,...","[Barrios, interminables, ciudad, fin, hace, su...","[Barrios: PROPN ROOT, interminable: ADJ flat, ...",NEG,0.887432,0.103598,0.008971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,72,47. Barrio Sur_.docx,BARRIO SUR,ZITARROZA Y ESTRAZULAS,MONTEVIDEO,1970,MILONGA?,47,ALFREDO ZITARROZA,MALE,...,"['gorrión', 'que', 'arrulla', 'un', 'sueño', '...","[gorrión, arrulla, sueño, tarde, cae, gatos, h...","[altillo, amarillos, amor, amándote, angostos,...","(Gorrión, que, arrulla, un, sueño, tarde, que,...","[Gorrión, arrulla, sueño, tarde, cae, gatos, h...","[Gorrión: PROPN nsubj, arrullar: VERB acl, sue...",NEU,0.314978,0.579061,0.105961
73,73,30. Montevideo_.docx,MONTEVIDEO,TABARE CARDOZO,MONTEVIDEO,2008,MURGA,30,TABARE CARDOZO,MALE,...,"['vengo', 'de', 'un', 'sitio', 'perdido', 'en'...","[vengo, sitio, perdido, sur, gallegos, tanos, ...","[acunó, africano, amor, bandoneón, barco, bohe...","(Vengo, de, un, sitio, perdido, en, el, sur, e...","[Vengo, sitio, perdido, sur, gallegos, tanos, ...","[venir: VERB ROOT, sitio: NOUN obl, perdido: A...",NEU,0.041060,0.648363,0.310578
74,74,27. Ya comienza.docx,YA COMIENZA,JOSE CARBAJAL,MONTEVIDEO,1972,CANDOMBE,27,JOSE CARBAJAL,MALE,...,"['callecitas', 'de', 'adoquines', 'te', 'harán...","[callecitas, adoquines, harán, vibrar, canto, ...","[abanicos, abuelas, adoquines, almidones, amon...","(Callecitas, de, adoquines, te, harán, vibrar,...","[Callecitas, adoquines, harán, vibrar, canto, ...","[callecita: NOUN nsubj, adoquín: NOUN nmod, ha...",NEG,0.75884

In [130]:
all_df['Place'].unique().tolist()

['BUENOS AIRES', 'MADRID', 'MONTEVIDEO']

In [4]:
buenos_aires_df.shape, madrid_df.shape, montevideo_df.shape

((94, 13), (79, 13), (77, 13))

In [5]:
list = buenos_aires_df["words"][0].replace("'","").replace("[","").replace(" ", "").replace("]","").split(",")
list

['sale',
 'un',
 'bote',
 'a',
 'sufrir',
 'no',
 'es',
 'como',
 'venecia',
 'sin',
 'ti',
 'camalote',
 'va',
 'flotando',
 'arrastrado',
 'sin',
 'poder',
 'nadar',
 'y',
 'tapa',
 '"pa"',
 'tapar',
 'ignorar',
 'gentinar',
 'humano',
 'en',
 'el',
 'lecho',
 'está',
 'nacido',
 'del',
 'sudor',
 'va',
 '"patras"',
 'patriota',
 'petitero',
 'llora',
 'el',
 'sauce',
 'sin',
 'talar',
 'chupa',
 'atucha',
 'al',
 'río',
 'su',
 'canción',
 'gaucho',
 'anfibio',
 'en',
 'junín',
 'cama',
 'escama',
 'en',
 'federación',
 'y',
 'tapa',
 '"pa"',
 'tapar',
 'ignorar',
 'gentinar',
 'humano',
 'en',
 'el',
 'lecho',
 'está',
 'nacido',
 'del',
 'sudor',
 'va',
 '"patrás"',
 'patriota',
 'petitero',
 'como',
 'en',
 'melincue',
 'agua',
 'en',
 'buenos',
 'aires',
 'como',
 'en',
 'federación',
 'agua',
 'en',
 'buenos',
 'aires',
 'sale',
 'un',
 'bote',
 'a',
 'sufrir',
 'no',
 'es',
 'como',
 'venecia',
 'sin',
 'ti',
 'camalote',
 'va',
 'flotando',
 'arrastrado',
 'sin',
 'poder',
 '

In [6]:
import nltk
# Stanza for sentiment analysis

stopwords = nltk.corpus.stopwords.words('spanish')
stopwords.append(" ")
stopwords.append(",")
stopwords.append(":")
stopwords.append(";")
stopwords.append(".")
stopwords.append("!")
stopwords.append("?")
stopwords.append("'")
stopwords.append("¿")


# filtered_text = [t for t in tokens if not t in stopwords.words("english")]

buenos_aires_df["content_words"] = buenos_aires_df["words"].apply(lambda x: [item for item in str(x).replace("'","").replace("[","").replace(" ", "").replace("]","").split(",") if item not in stopwords])
madrid_df["content_words"] = madrid_df["words"].apply(lambda x: [item for item in str(x).replace("'","").replace("[","").replace(" ", "").replace("]","").split(",") if item not in stopwords])
montevideo_df["content_words"] = montevideo_df["words"].apply(lambda x: [item for item in str(x).replace("'","").replace("[","").replace(" ", "").replace("]","").split(",") if item not in stopwords])

In [7]:
from collections import OrderedDict


madrid_df["unique_words"] = madrid_df["content_words"].apply(lambda x: np.delete(np.unique(x), np.where(np.unique(x) == '')))
buenos_aires_df["unique_words"] = buenos_aires_df["content_words"].apply(lambda x: np.delete(np.unique(x), np.where(np.unique(x) == '')))
montevideo_df["unique_words"] = montevideo_df["content_words"].apply(lambda x: np.delete(np.unique(x), np.where(np.unique(x) == '')))

In [9]:
# Need to count number of times a word appears across songs
from collections import Counter

uniq_words = madrid_df["unique_words"].to_numpy()
madrid_uniq_words = np.hstack(uniq_words)
word_count = Counter(madrid_uniq_words)
word_count = sorted(word_count.items(), key=lambda item: (-item[1], item[0]))
madrid_words = OrderedDict(word_count)

In [10]:

all_words = []
uniq_words = montevideo_df["unique_words"].to_numpy()
monte_uniq_words = np.hstack(uniq_words)
word_count = Counter(monte_uniq_words)
word_count = sorted(word_count.items(), key=lambda item: (-item[1], item[0]))
montevideo_words = OrderedDict(word_count)
montevideo_words

OrderedDict([('siempre', 25),
             ('si', 24),
             ('mar', 21),
             ('montevideo', 20),
             ('ciudad', 19),
             ('gente', 19),
             ('noche', 19),
             ('va', 17),
             ('barrio', 16),
             ('vez', 16),
             ('calle', 15),
             ('corazón', 15),
             ('hoy', 15),
             ('tiempo', 15),
             ('vida', 15),
             ('van', 14),
             ('voy', 14),
             ('día', 13),
             ('igual', 13),
             ('vieja', 13),
             ('allá', 12),
             ('viejo', 12),
             ('viento', 12),
             ('alma', 11),
             ('bien', 11),
             ('hace', 11),
             ('luna', 11),
             ('nunca', 11),
             ('sol', 11),
             ('tan', 11),
             ('vivir', 11),
             ('amor', 10),
             ('aquí', 10),
             ('cada', 10),
             ('da', 10),
             ('lugar', 10),
             

In [11]:

all_words = []
uniq_words = buenos_aires_df["unique_words"].to_numpy()
ba_uniq_words = np.hstack(uniq_words)
word_count = Counter(ba_uniq_words)
word_count = sorted(word_count.items(), key=lambda item: (-item[1], item[0]))
buenos_aires_words = OrderedDict(word_count)
buenos_aires_words

OrderedDict([('aires', 50),
             ('buenos', 49),
             ('si', 37),
             ('vos', 33),
             ('bien', 29),
             ('ciudad', 27),
             ('hoy', 27),
             ('tan', 27),
             ('voy', 26),
             ('amor', 25),
             ('noche', 25),
             ('calle', 24),
             ('nunca', 24),
             ('quiero', 24),
             ('va', 24),
             ('vida', 24),
             ('sol', 23),
             ('ser', 22),
             ('siempre', 22),
             ('calles', 21),
             ('ver', 20),
             ('hace', 19),
             ('sólo', 18),
             ('vez', 18),
             ('sé', 17),
             ('tango', 17),
             ('cada', 16),
             ('corrientes', 16),
             ('gente', 16),
             ('día', 15),
             ('mundo', 15),
             ('nadie', 15),
             ('tiempo', 15),
             ('alma', 14),
             ('cielo', 14),
             ('corazón', 14),
            

In [12]:
import nltk
from collections import OrderedDict
from collections import Counter

# stopwords = nltk.corpus.stopwords.words('spanish')

# frames = [buenos_aires_df, montevideo_df, madrid_df]
# all_df = pd.concat(frames)
# all_df["content_words"] = all_df["words"].apply(lambda x: [item for item in str(x).replace("'","").replace("[","").replace(" ", "").replace("]","").split(",") if item not in stopwords])
# all_df["unique_words"] = all_df["content_words"].apply(lambda x: np.delete(np.unique(x), np.where(np.unique(x) == '')))

In [13]:
uniq_words = all_df["unique_words"].to_numpy()
all_uniq_words = np.hstack(uniq_words)
word_count = Counter(all_uniq_words)
word_count = sorted(word_count.items(), key=lambda item: (-item[1], item[0]))
all_words = OrderedDict(word_count)
all_words

OrderedDict([('si', 91),
             ('siempre', 72),
             ('ciudad', 63),
             ('noche', 62),
             ('aquí', 59),
             ('gente', 55),
             ('hoy', 55),
             ('madrid', 55),
             ('voy', 55),
             ('aires', 54),
             ('va', 54),
             ('vida', 54),
             ('sol', 53),
             ('buenos', 51),
             ('tan', 51),
             ('vez', 51),
             ('bien', 48),
             ('calle', 48),
             ('calles', 47),
             ('amor', 46),
             ('corazón', 45),
             ('nunca', 44),
             ('ser', 44),
             ('vos', 43),
             ('quiero', 39),
             ('barrio', 38),
             ('día', 38),
             ('cada', 37),
             ('hace', 37),
             ('mar', 37),
             ('tiempo', 37),
             ('sé', 35),
             ('toda', 35),
             ('cielo', 34),
             ('nadie', 34),
             ('van', 34),
             ('al

In [14]:
cities = all_df['Place'].unique()
city_words = []
for city in cities:
    city_df = all_df.loc[all_df['place'] == city]
    uniq_words = city_df["unique_words"].to_numpy()
    uniq_words = np.hstack(uniq_words)
    word_count = Counter(uniq_words)
    word_count = sorted(word_count.items(), key=lambda item: (-item[1], item[0]))
    all_words = OrderedDict(word_count)
    city_words.append(all_words)

[OrderedDict([('aires', 50),
              ('buenos', 49),
              ('si', 37),
              ('vos', 33),
              ('bien', 29),
              ('ciudad', 27),
              ('hoy', 27),
              ('tan', 27),
              ('voy', 26),
              ('amor', 25),
              ('noche', 25),
              ('calle', 24),
              ('nunca', 24),
              ('quiero', 24),
              ('va', 24),
              ('vida', 24),
              ('sol', 23),
              ('ser', 22),
              ('siempre', 22),
              ('calles', 21),
              ('ver', 20),
              ('hace', 19),
              ('sólo', 18),
              ('vez', 18),
              ('sé', 17),
              ('tango', 17),
              ('cada', 16),
              ('corrientes', 16),
              ('gente', 16),
              ('día', 15),
              ('mundo', 15),
              ('nadie', 15),
              ('tiempo', 15),
              ('alma', 14),
              ('cielo', 14),
       

In [15]:
all_df.loc[all_df['place'] == "MADRID"]["text"][0]

'Antes de marcharme miraré las calles dentro de un autobús los Jerónimos la cuesta de Moyano y toda la linea azul La última mirada desde aquí me hará saltar al cielo de Madrid la última mirada es para ti Retiro, Atocha, Goya y Chamberí Antes de apagarme compraré en el rastro subiré al Pirulí llevaré conmigo al oso y al madroño dormiré en Chamartín la última mirada es frente a Sol hasta llegar a la Plaza Mayor la última mirada desde aquí será saltar al cielo y ver Madrid Antes de morirme elegiré una nube me uniré al temporal y tras elevarme lloraré una lluvia y mojaré la ciudad La última mirada desde allá Pudiera ser la calle de Alcalá La última mirada desde allí Por un agujerito ver Madrid La última mirada desde allá Pudiera ser la calle de Alcalá La última mirada desde allí Por un agujerito ver Madrid Por un agujerito ver Madrid Por un agujerito ver Madrid Por un agujerito ver Madrid Por un agujerito ver Madrid Por un agujerito ver Madrid Por un agujerito ver Madrid Por un agujerito v

In [16]:
all_df["text"][0]

0    Sale un bote a sufrir No es como Venecia sin t...
0    A dónde se han ido mis dieciocho años La prime...
0    Antes de marcharme miraré las calles dentro de...
Name: text, dtype: object

In [17]:
import spacy
from spacy import displacy

In [18]:
# !python -m spacy download es_core_news_md

In [19]:
nlp = spacy.load('es_core_news_md')
pickle.dump( nlp, open( "pos_tagger.p", "wb" ) )

C:\Users\joana\anaconda3\lib\site-packages\spacy\util.py:887: UserWarning: [W095] Model 'es_core_news_md' (3.4.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.5.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [20]:
pos = []
for text in all_df["text"]:
    doc = nlp(text)
    pos.append(doc)

In [21]:
for document in pos:
    for token in document:
        print(token.lemma_, token.pos_, token.dep_)

salir VERB advcl
uno DET det
bote NOUN obj
a ADP mark
sufrir VERB acl
no ADV advmod
ser AUX cop
como SCONJ mark
Venecia PROPN nsubj
sin ADP case
tú PRON obl
  SPACE dep
Camalote PROPN flat
ir AUX aux
flotar VERB advcl
arrastrado ADJ obj
sin ADP mark
poder AUX aux
nadar VERB advcl
Y CCONJ advmod
tapa NOUN conj
pa ADP case
' SYM obl
tapar VERB xcomp
ignorar VERB xcomp
gentinar NOUN xcomp
Humano PROPN obj
en ADP case
el DET det
lecho NOUN obl
estar AUX cop
nacido ADJ advcl
del ADP case
sudor NOUN obl
Va PROPN advcl
pa'tra NOUN obj
patriota ADJ amod
petitero ADJ advcl
  SPACE dep
lloro ADJ flat
el DET det
sauce NOUN obj
sin ADP mark
talar VERB acl
Chupa PROPN obj
Atucha PROPN flat
al ADP case
río NOUN obl
su DET det
canción NOUN obj
Gaucho PROPN appos
anfibio NOUN amod
en ADP case
Junín PROPN nmod
Cama PROPN flat
escama ADJ amod
en ADP case
Federación PROPN obl
  SPACE dep
Y CCONJ cc
tapa PROPN appos
pa ADP case
' SYM punct
tapar VERB csubj
ignorar VERB xcomp
gentinar NOUN xcomp
Humano PRO

el DET det
escuela NOUN obl
  SPACE dep
festejamos VERB advcl
fiesta NOUN obj
de ADP case
otro DET det
tierra NOUN nmod
  SPACE dep
el DET det
indio NOUN nsubj
que PRON nsubj
matar VERB acl
Rivera PROPN nsubj
  SPACE dep
pelear VERB advcl
junto ADJ case
a ADP fixed
Artigas PROPN obl
  SPACE dep
Contra ADP case
el DET det
gringado ADJ advmod
veo VERB advcl
tu DET det
careta NOUN obj
idiota ADJ amod
salir VERB advcl
del ADP case
Shopping PROPN obj
  SPACE dep
ver VERB advcl
a ADP case
Rivera PROPN obj
montado ADJ obj
sobre ADP case
su DET det
caballo NOUN obl
  SPACE dep
frente NOUN case
a ADP fixed
él PRON obl
el DET det
cruz PROPN obj
blanca PROPN flat
del ADP case
poder NOUN nmod
Pa’l PROPN appos
extranjero NOUN flat
nuestro DET det
tierra NOUN flat
siempre ADV advmod
ser AUX cop
barato ADJ advcl
  SPACE dep
él PRON obj
pagar VERB compound
con ADP case
todo DET det
su DET det
mierdita NOUN obl
de ADP case
marca NOUN nmod
  SPACE dep
Frívolas PROPN nmod
religión NOUN flat
gringo ADJ am

In [22]:
all_df["pos"] = pos 

In [23]:
all_df["pos"][0]
frames = [buenos_aires_df, montevideo_df, madrid_df]
all_df["pos"] = pos 
all_df["content_pos"] = all_df["pos"].apply(lambda x: [item for item in x if str(item) not in stopwords])
# [item for item in str(x).replace("'","").replace("[","").replace(" ", "").replace("]","").split(",") if item not in stopwords]

In [24]:
all_df.loc[all_df['place'] == "BUENOS AIRES"]["content_pos"][0]

[Sale,
 bote,
 sufrir,
 No,
 Venecia,
 Camalote,
 va,
 flotando,
 Arrastrado,
 poder,
 nadar,
 Y,
 tapa,
 pa,
 tapar,
 ignorar,
 gentinar,
 Humano,
 lecho,
 Nacido,
 sudor,
 Va,
 pa'tras,
 patriota,
 petitero,
 Llora,
 sauce,
 talar,
 Chupa,
 Atucha,
 río,
 canción,
 Gaucho,
 anfibio,
 Junín,
 Cama,
 escama,
 Federación,
 Y,
 tapa,
 pa,
 tapar,
 ignorar,
 gentinar,
 Humano,
 lecho,
 Nacido,
 sudor,
 Va,
 pa'trás,
 patriota,
 petitero,
 Como,
 Melincue,
 Agua,
 Buenos,
 Aires,
 Como,
 Federación,
 Agua,
 Buenos,
 Aires,
 Sale,
 bote,
 sufrir,
 No,
 Venecia,
 Camalote,
 va,
 flotando,
 Arrastrado,
 poder,
 nadar,
 Y,
 tapa,
 pa,
 tapar,
 ignorar,
 gentinar,
 Humano,
 lecho,
 Nacido,
 sudor,
 Va,
 pa'tras,
 patriota,
 petitero,
 Como,
 Melincue,
 Agua,
 Buenos,
 Aires,
 Como,
 Federación,
 Agua,
 Buenos,
 Aires,
 Agua,
 Buenos,
 Aires,
 Agua,
 Buenos,
 Aires,
 Agua,
 Buenos,
 Aires]

In [25]:
all_df["POS"] = all_df["content_pos"].apply(lambda x: [str(item.lemma_ + ": " + item.pos_ + " " + item.dep_)  for item in x])

In [26]:
all_df["POS"].apply(lambda x: [print(item, type(item)) for item in x])

salir: VERB advcl <class 'str'>
bote: NOUN obj <class 'str'>
sufrir: VERB acl <class 'str'>
no: ADV advmod <class 'str'>
Venecia: PROPN nsubj <class 'str'>
Camalote: PROPN flat <class 'str'>
ir: AUX aux <class 'str'>
flotar: VERB advcl <class 'str'>
arrastrado: ADJ obj <class 'str'>
poder: AUX aux <class 'str'>
nadar: VERB advcl <class 'str'>
Y: CCONJ advmod <class 'str'>
tapa: NOUN conj <class 'str'>
pa: ADP case <class 'str'>
tapar: VERB xcomp <class 'str'>
ignorar: VERB xcomp <class 'str'>
gentinar: NOUN xcomp <class 'str'>
Humano: PROPN obj <class 'str'>
lecho: NOUN obl <class 'str'>
nacido: ADJ advcl <class 'str'>
sudor: NOUN obl <class 'str'>
Va: PROPN advcl <class 'str'>
pa'tra: NOUN obj <class 'str'>
patriota: ADJ amod <class 'str'>
petitero: ADJ advcl <class 'str'>
lloro: ADJ flat <class 'str'>
sauce: NOUN obj <class 'str'>
talar: VERB acl <class 'str'>
Chupa: PROPN obj <class 'str'>
Atucha: PROPN flat <class 'str'>
río: NOUN obl <class 'str'>
canción: NOUN obj <class 'str'>
G

0     [None, None, None, None, None, None, None, Non...
1     [None, None, None, None, None, None, None, Non...
2     [None, None, None, None, None, None, None, Non...
3     [None, None, None, None, None, None, None, Non...
4     [None, None, None, None, None, None, None, Non...
                            ...                        
74    [None, None, None, None, None, None, None, Non...
75    [None, None, None, None, None, None, None, Non...
76    [None, None, None, None, None, None, None, Non...
77    [None, None, None, None, None, None, None, Non...
78    [None, None, None, None, None, None, None, Non...
Name: POS, Length: 250, dtype: object

In [27]:
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="es")
pickle.dump( analyzer, open( "sent_analyzer.p", "wb" ) )

sentiment_out = []
sentiment_neg = []
sentiment_neu = []
sentiment_pos = []

for text in all_df["text"]:
    res = analyzer.predict(text)
    sentiment_out.append(res.output)
    sentiment_neg.append(res.probas["NEG"])
    sentiment_neu.append(res.probas["NEU"])
    sentiment_pos.append(res.probas["POS"])

loading configuration file config.json from cache at C:\Users\joana/.cache\huggingface\hub\models--pysentimiento--robertuito-sentiment-analysis\snapshots\12e030859ce19539e24b486ac84ffebb9b68ecf1\config.json
Model config RobertaConfig {
  "_name_or_path": "pysentimiento/robertuito-sentiment-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NEG",
    "1": "NEU",
    "2": "POS"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NEG": 0,
    "NEU": 1,
    "POS": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type":

In [28]:
# sentiment_df = pd.DataFrame(sentiment)
# sentiment_df
sentiment_out

['NEG',
 'NEU',
 'NEU',
 'NEU',
 'NEG',
 'NEG',
 'NEU',
 'NEG',
 'NEU',
 'POS',
 'POS',
 'NEU',
 'POS',
 'NEG',
 'NEG',
 'NEG',
 'NEU',
 'POS',
 'NEG',
 'NEG',
 'NEU',
 'NEG',
 'POS',
 'NEG',
 'NEG',
 'POS',
 'NEG',
 'NEU',
 'NEU',
 'POS',
 'NEU',
 'NEU',
 'POS',
 'NEG',
 'NEU',
 'NEU',
 'NEG',
 'NEU',
 'NEG',
 'POS',
 'NEG',
 'NEU',
 'NEU',
 'POS',
 'NEU',
 'NEU',
 'NEU',
 'POS',
 'NEU',
 'NEG',
 'POS',
 'NEG',
 'NEG',
 'POS',
 'NEG',
 'POS',
 'NEU',
 'NEG',
 'NEG',
 'NEU',
 'NEG',
 'NEU',
 'NEG',
 'NEG',
 'NEG',
 'NEG',
 'NEG',
 'NEU',
 'NEU',
 'NEG',
 'NEU',
 'NEG',
 'NEU',
 'POS',
 'NEU',
 'NEG',
 'POS',
 'NEU',
 'NEU',
 'NEG',
 'POS',
 'NEG',
 'NEU',
 'NEU',
 'NEG',
 'POS',
 'NEG',
 'NEG',
 'NEG',
 'POS',
 'NEG',
 'NEU',
 'NEG',
 'NEU',
 'NEU',
 'NEU',
 'NEU',
 'POS',
 'NEG',
 'NEU',
 'NEG',
 'NEG',
 'NEU',
 'NEU',
 'NEG',
 'POS',
 'NEU',
 'NEU',
 'POS',
 'NEG',
 'NEG',
 'NEG',
 'NEG',
 'NEG',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'NEU',
 'NEU',
 'NEU',
 'NEU',
 'NEU',
 'NEG',


In [30]:
all_df['Sentiment_Output'] = sentiment_out
all_df['Sentiment_Neg_Probability'] = sentiment_neg
all_df['Sentiment_Neu_Probability'] = sentiment_neu
all_df['Sentiment_Pos_Probability'] = sentiment_pos

In [31]:
trimmed = all_df.drop(['lyrics','pos','content_pos','words'], axis=1)
trimmed

,Unnamed: 0,file,name,singer,place,year,genre,lyricnumber,composer,sex,text,content_words,unique_words,POS,Sentiment_Output,Sentiment_Neg_Probability,Sentiment_Neu_Probability,Sentiment_Pos_Probability
0,0,B.A. 51 Agua en Buenos Aires.docx,AGUA EN BUENOS AIRES,DIVIDIDOS,BUENOS AIRES,1995,ROCK-POP?,51,DIEGO ARNERO-RICARDO MOLLO,MALE,Sale un bote a sufrir No es como Venecia sin t...,"[sale, bote, sufrir, venecia, camalote, va, fl...","[""pa"", ""patras"", ""patrás"", agua, aires, anfibi...","[salir: VERB advcl, bote: NOUN obj, sufrir: VE...",NEG,0.732685,0.250028,0.017287
1,1,B.A. 58 Idolo de los quemados.docx,IDOLO DE LOS QUEMADOS,LEON GIECO,BUENOS AIRES,2001,ROCK,58,LEON GIECO,MALE,"Soy mezcla de baguala, internet y tango viejo ...","[mezcla, baguala, , internet, tango, viejo, he...","[""andá, 2001, 3000, 411, aconsejó, aires, alas...","[ser: AUX cop, mezcla: NOUN ROOT, baguala: NOU...",NEU,0.416313,0.519488,0.064199
2,2,B.A. 25 Con la frente marchita.docx,CON LA FRENTE MARCHITA,JOAQUIN SABINA,BUENOS AIRES,1990,POP-BANDONEON?,25,JOAQUIN SABINA,MALE,Sentados en corro Merendábamos besos y porros ...,"[sentados, corro, merendábamos, besos, porros,...","[80, adiós, agüita, aires, amor, andaluz, aque...","[sentado: ADJ nsubj, corro: NOUN advcl, Merend...",NEU,0.200022,0.635468,0.164511
3,3,B.A. 15 Los locos de Buenos Aires.docx,LOS LOCOS DE BUENOS AIRES,ALEJANDRO DEL PRADO,BUENOS AIRES,1984,POP,15,ALEJANDRO DEL PRADO,MALE,"Uno está solo y espera, Otro cree a su manera,...","[solo, espera, , cree, manera, , ciego, locura...","[""40"", actor, aires, ama, amenazante, andan, a...","[Uno: PROPN nsubj, solo: ADJ advcl, esperar: V...",NEU,0.475670,0.490647,0.033683
4,4,B.A. 65 Oeste Cercano.docx,OESTE CERCANO,ANTONIO BIRABENT,BUENOS AIRES,2005,POP,65,ANTONIO BIRABENT,MALE,"Barrios interminables, la ciudad no tiene fin...","[barrios, interminables, , ciudad, fin, hace, ...","[adivino., aldea, almohada, arteria, asfalto.,...","[Barrios: PROPN ROOT, interminable: ADJ flat, ...",NEG,0.887432,0.103598,0.008971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,74,75. Vente pa Madrid.docx,VENTE PA MADRID,KETAMA,MADRID,1988,FLAMENCO-FUSION,75,KETAMA,MALE,Yo tenia un primo en Getafe Que se tuvo que ma...,"[tenia, primo, getafe, marchar, ir, alicante, ...","[""pa"", acuerda, alicante, allí, alma, antonio,...","[yo: PRON nsubj, teniar: VERB ROOT, primo: NOU...",NEG,0.695789,0.262327,0.041885
75,75,77. Ahi te dejo Madrid.docx,AHI TE DEJO MADRID,SHAKIRA,MADRID,2001,ROCK-POP,76,SHAKIRA,FEMALE,"Sí, ya es hora de esconder del mundo el dolor...","[, hora, esconder, mundo, dolor, bajo, piel, m...","[actuar, adiós, ah, ahi, ahí, anís, aquí, ay, ...","[sí: INTJ advcl, hora: NOUN ROOT, esconder: VE...",NEG,0.770895,0.206544,0.022562
76,76,78. Mi Madrid.docx,MI MADRID,MANUEL CARRASCO,MADRID,2019,POP,78,MANUEL CARRASCO,MALE,"Madrid, todas mis noches contigo Están present...","[madrid, , todas, noches, contigo, presentes, ...","[abiertos, acera, acordes, ambulancias, amor, ...","[Madrid: PROPN nsubj, todo: DET det, noche: NO...",POS,0.032936,0.278909,0.688156
77,77,20. Pobre Madrid.docx,POBRE MADRID,BARON ROJO,MADRID,1987,ROCK,20,JOSE LUIS CAMPUZANO,MALE,¿Dónde está aquel Madrid que de niño conocí? ¿...,"[¿dónde, aquel, madrid, niño, conocí?, ¿quién,...","[adiós, alquitrán, aquel, aquí, así, capital.,...","[dónde: PRON obl, aquel: DET det, Madrid: PROP...",NEG,0.968165,0.028593,0.003242


In [32]:
# Able to query the dataframe
# get labels from the sentiment, with the probabilities, count number of labels per city as well
# all_df.loc[all_df['place'] == "MONTEVIDEO"][["singer","name","sentiment"]][:20]
# import pickle
# all_df = all_df.rename({'Unnamed: 0': 'Index'}, axis=1)
# pickle.dump( trimmed, open( "lyrics_corpus.p", "wb" ) )
agua = all_df.loc[all_df['name'] == "AGUA EN BUENOS AIRES"]

In [33]:
agua["file"].item()
agua.columns

Index(['Unnamed: 0', 'file', 'name', 'singer', 'place', 'year', 'genre',
       'lyricnumber', 'composer', 'sex', 'lyrics', 'text', 'words',
       'content_words', 'unique_words', 'pos', 'content_pos', 'POS',
       'Sentiment_Output', 'Sentiment_Neg_Probability',
       'Sentiment_Neu_Probability', 'Sentiment_Pos_Probability'],
      dtype='object')

In [34]:
# all_df = pickle.load( open( "lyrics_corpus.p", "rb" ) )

In [35]:
uniq_words = all_df.loc[all_df['place'] == "BUENOS AIRES"]["unique_words"].to_numpy()
all_uniq_words = np.hstack(uniq_words)
word_count = Counter(all_uniq_words)
word_count = sorted(word_count.items(), key=lambda item: (-item[1], item[0]))
all_words = OrderedDict(word_count)
all_words

OrderedDict([('aires', 50),
             ('buenos', 49),
             ('si', 37),
             ('vos', 33),
             ('bien', 29),
             ('ciudad', 27),
             ('hoy', 27),
             ('tan', 27),
             ('voy', 26),
             ('amor', 25),
             ('noche', 25),
             ('calle', 24),
             ('nunca', 24),
             ('quiero', 24),
             ('va', 24),
             ('vida', 24),
             ('sol', 23),
             ('ser', 22),
             ('siempre', 22),
             ('calles', 21),
             ('ver', 20),
             ('hace', 19),
             ('sólo', 18),
             ('vez', 18),
             ('sé', 17),
             ('tango', 17),
             ('cada', 16),
             ('corrientes', 16),
             ('gente', 16),
             ('día', 15),
             ('mundo', 15),
             ('nadie', 15),
             ('tiempo', 15),
             ('alma', 14),
             ('cielo', 14),
             ('corazón', 14),
            

In [36]:
for place in all_df["place"].unique():
    print(place)

BUENOS AIRES
MONTEVIDEO
MADRID
